In [1]:
# Import necessary libraries
import logging
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial import cKDTree
from scipy.stats import zscore

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

from pathlib import Path

In [2]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Set Base Directory
base_dir = Path.cwd().parent  # Moves one level up from current working directory

# Data Directory
data_dir = base_dir / '02_Data'

In [4]:
# Load datasets
train = pd.read_csv(data_dir / "Train.csv")
test = pd.read_csv(data_dir / "Test.csv")
toilets = pd.read_csv(data_dir / "toilets.csv")
waste_management = pd.read_csv(data_dir / "waste_management.csv")
water_sources = pd.read_csv(data_dir / "water_sources.csv")

# EDA

### Non-null counts and Dtypes

In [ ]:
print("Train Data Info\n")
train.info()

In [ ]:
print("Test Data Info\n")
test.info()

### Number of Missing Values

In [ ]:
print("Train Data - Number of Missing Values\n")
train.isnull().sum()

In [ ]:
print("Test Data - Number of Missing Values\n")
test.isnull().sum()

### Duplicates

In [ ]:
duplicate_count = train.duplicated().sum()
print(duplicate_count)

In [ ]:
duplicate_count_test = test.duplicated().sum()
print(duplicate_count_test)

### Outliers

In [ ]:
# Function to plot boxplots for numerical features
def plot_boxplots(df, title):
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df.select_dtypes(include=['number']))
    plt.xticks(rotation=45)
    plt.title(title)
    plt.show()

# Boxplots
plot_boxplots(train, "Boxplot of Numerical Features (Train Dataset)")
plot_boxplots(test, "Boxplot of Numerical Features (Test Dataset)")


### Outliers Using IQR

In [12]:
def detect_outliers_iqr(data, multiplier=1.5):
    """Detects outliers in a given Pandas Series using the IQR method."""
    q1 = np.percentile(data.dropna(), 25)
    q3 = np.percentile(data.dropna(), 75)
    IQR = q3 - q1
    lwr_bound = q1 - (1.5 * IQR)
    upr_bound = q3 + (multiplier * IQR)

    return data[(data < lwr_bound) | (data > upr_bound)].index

### Summary Statistics

In [ ]:
print("Train Data - Summary Stats\n")

train.describe()

In [ ]:
print("Test Data - Summary Stats\n")

test.describe()

### Variable Distribution

In [15]:
# # Plot target variable distribution
# plt.figure(figsize=(10, 5))
# sns.histplot(train['Total'], bins=30, kde=True)
# plt.title("Distribution of Total Variable")
# plt.show()

### Correlation in Train Data

In [16]:
# # Select only numeric columns
# numeric_cols = train.select_dtypes(include=['number'])

# # Compute correlation only for numeric features
# plt.figure(figsize=(12, 6))
# sns.heatmap(numeric_cols.corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
# plt.title("Feature Correlation Heatmap")
# plt.show()


# Preprocessing

### Missing Values

In [ ]:
# Imputing with Median
train.loc[:, 'Total'] = train['Total'].fillna(train['Total'].median())

# Number of Missing Values
train.isnull().sum()

### Duplicates, Skewness and Outliers

In [18]:
train_with_duplicates = train.copy()

In [19]:
# Log_Total before detecting outliers
train_with_duplicates["Log_Total"] = np.log1p(train_with_duplicates["Total"])

train_with_duplicates["Z_Score_Total"] = np.abs(zscore(train_with_duplicates["Total"]))
train_with_duplicates["Z_Score_Log_Total"] = np.abs(zscore(train_with_duplicates["Log_Total"]))

### Outlier Detection Before Removing Duplicates

In [20]:
# Using IQR
outliers_iqr_with_duplicates = train_with_duplicates.loc[detect_outliers_iqr(train_with_duplicates["Total"])]
outliers_iqr_log_with_duplicates = train_with_duplicates.loc[detect_outliers_iqr(train_with_duplicates["Log_Total"])]


# Using Z-score
outliers_zscore_with_duplicates = train_with_duplicates.loc[train_with_duplicates["Z_Score_Total"] > 3]
outliers_zscore_log_with_duplicates = train_with_duplicates.loc[train_with_duplicates["Z_Score_Log_Total"] > 3]


### Removing Duplicates

In [21]:
# Drop duplicate rows
train = train.drop_duplicates()

### Outlier Detection After Removing Duplicates

In [22]:
# Log_Total after dropping duplicates
train["Log_Total"] = np.log1p(train["Total"])

train["Z_Score_Total"] = np.abs(zscore(train["Total"]))
train["Z_Score_Log_Total"] = np.abs(zscore(train["Log_Total"]))

In [23]:
# Detect outliers using IQR on Total and Log_Total after dropping duplicates
outliers_iqr_no_duplicates = train.loc[detect_outliers_iqr(train["Total"])]
outliers_iqr_log_no_duplicates = train.loc[detect_outliers_iqr(train["Log_Total"])]


# Detect outliers using Z-score on Total and Log_Total after dropping duplicates
outliers_zscore_no_duplicates = train.loc[train["Z_Score_Total"] > 3]
outliers_zscore_log_no_duplicates = train.loc[train["Z_Score_Log_Total"] > 3]

In [ ]:
outlier_df = pd.DataFrame(
    {
        "Method": ["IQR", "Z-Score"],
        "Total_With_Duplicates": [outliers_iqr_with_duplicates.shape[0], outliers_zscore_with_duplicates.shape[0]],
        "Total_No_Duplicates": [outliers_iqr_no_duplicates.shape[0], outliers_zscore_no_duplicates.shape[0]],
        "Log_Total_With_Duplicates": [outliers_iqr_log_with_duplicates.shape[0], outliers_zscore_log_with_duplicates.shape[0]],
        "Log_Total_No_Duplicates": [outliers_iqr_log_no_duplicates.shape[0], outliers_zscore_log_no_duplicates.shape[0]]
    }
)


print("Number of Outliers With and Without Duplicates in Total and Log_Total")

outlier_df

### Train Data Set (Review)

In [ ]:
train.head()

In [26]:
# train = train.drop(["Z_Score_Total","Z_Score_Log_Total"], axis=1)
# train.head()

### Impute Outliers with Median

In [27]:
train_no_outliers = train.copy()

In [28]:
# Impute IQR outliers with median
for col in ["Total", "Log_Total"]:
    median_value = train_no_outliers[col].median()
    train_no_outliers.loc[detect_outliers_iqr(train_no_outliers[col]), col] = median_value

# Impute Z-score outliers with median
for col in ["Total", "Log_Total"]:
    median_value = train_no_outliers[col].median()
    train_no_outliers.loc[train_no_outliers[f"Z_Score_{col}"] > 3, col] = median_value


In [ ]:
# Step 1: Detect Outliers Again After Imputation
outliers_iqr_after = {
    "Total": len(detect_outliers_iqr(train_no_outliers["Total"])),
    "Log_Total": len(detect_outliers_iqr(train_no_outliers["Log_Total"]))
}

outliers_zscore_after = {
    "Total": (train_no_outliers["Z_Score_Total"] > 3).sum(),
    "Log_Total": (train_no_outliers["Z_Score_Log_Total"] > 3).sum()
}

# Step 2: Create DataFrame to Compare Outliers Before and After Imputation
outliers_after_df = pd.DataFrame(
    {
        "Method": ["IQR", "Z-Score"],
        "Total_After_Imputation": [outliers_iqr_after["Total"], outliers_zscore_after["Total"]],
        "Log_Total_After_Imputation": [outliers_iqr_after["Log_Total"], outliers_zscore_after["Log_Total"]]
    }
)

# Step 3: Print Results
print("Outliers Remaining After Imputation:")
print(outliers_after_df)


### More Preprocessing - Merging

In [30]:
# Backup
train_backup = train.copy()
test_backup = test.copy()

In [ ]:
# Check column names for all datasets
print("Train Columns:", train.columns)
print("Test Columns:", test.columns)
print("Toilets Columns:", toilets.columns)
print("Waste Management Columns:", waste_management.columns)
print("Water Sources Columns:", water_sources.columns)

#### Next Steps
- I'll merge train and test with the other datasets.
- Before that, I'll add a prefix like the starter notebook but instead of using the engineered "Month_Year_lat_lon", I'll take the orginal columns.

In [ ]:
# Function to add prefixes while keeping essential merge keys unchanged
def add_prefix(df, prefix, exclude_cols=["Location", "Year", "Month", "Transformed_Latitude", "Transformed_Longitude"]):
    """Renames columns by adding a prefix while keeping essential merge keys unchanged."""
    df = df.rename(columns={col: f"{prefix}_{col}" for col in df.columns if col not in exclude_cols})
    return df

# Apply prefixes to each auxiliary dataset
toilets_prefixed = add_prefix(toilets, "toilet")
waste_management_prefixed = add_prefix(waste_management, "waste")
water_sources_prefixed = add_prefix(water_sources, "water")

# Verify changes
print("Toilets Columns:", toilets_prefixed.columns)
print("Waste Management Columns:", waste_management_prefixed.columns)
print("Water Sources Columns:", water_sources_prefixed.columns)

## Start modeling

#### Make predictions on test